In [1]:
import pandas as pd
import numpy as np
import json
import random

try:
    pd.options.display.float_format = '{:.2f}'.format

    with open('../ex02/auto.json', 'r') as f:
        data = json.load(f)
        df = pd.DataFrame(data)
        print("Исходный датафрейм из JSON:")
        print(df.head())
        print(f"Размер исходного датафрейма: {df.shape}")

except FileNotFoundError as e:
    print("File not found")
    raise e

Исходный датафрейм из JSON:
      CarNumber  Refund   Fines    Make  Model
0  Y163O8161RUS    2.00 3200.00    Ford  Focus
1   E432XX77RUS    1.00 6500.00  Toyota  Camry
2   7184TT36RUS    1.00 2100.00    Ford  Focus
3  X582HE161RUS    2.00 2000.00    Ford  Focus
4  92918M178RUS    1.00 5700.00    Ford  Focus
Размер исходного датафрейма: (725, 5)


In [2]:
# Создаем выборку из 200 новых наблюдений
unique_cars = df[['CarNumber', 'Make', 'Model']].drop_duplicates()
print(f"\nКоличество уникальных автомобилей: {len(unique_cars)}")

# Создаем выборку из 200 наблюдений с random_state=21
np.random.seed(21)
random.seed(21)

# Для создания выборки используем существующие комбинации CarNumber, Make, Model
sample_size = 200
sample_indices = np.random.choice(len(unique_cars), size=sample_size, replace=True)
sample_cars = unique_cars.iloc[sample_indices].reset_index(drop=True)

# Добавляем случайные значения Refund и Fines из исходного датафрейма
refund_values = df['Refund'].dropna().values
fines_values = df['Fines'].dropna().values

sample_refunds = np.random.choice(refund_values, size=sample_size)
sample_fines = np.random.choice(fines_values, size=sample_size)

# Создаем датафрейм выборки
sample_df = pd.DataFrame({
    'CarNumber': sample_cars['CarNumber'],
    'Refund': sample_refunds,
    'Fines': sample_fines,
    'Make': sample_cars['Make'],
    'Model': sample_cars['Model']
})

print("\nВыборка из 200 наблюдений:")
print(sample_df.head())


Количество уникальных автомобилей: 532

Выборка из 200 наблюдений:
      CarNumber  Refund    Fines  Make  Model
0   T6329O50RUS    1.00 12300.00  Ford  Focus
1   H917TC36RUS    1.00 24200.00  Ford  Focus
2  9763HY161RUS    2.00  5800.00  Ford  Focus
3  H003MH197RUS    2.00  2100.00  Ford  Focus
4  O64097197RUS    2.00  2000.00  Ford  Focus


In [3]:
# Объединяем исходный датафрейм с выборкой
concat_rows = pd.concat([df, sample_df], ignore_index=True)
print(f"\nРазмер объединенного датафрейма: {concat_rows.shape}")

# 4. Добавляем колонку Year с случайными годами от 1980 до 2019
np.random.seed(21)
years = np.random.randint(1980, 2020, size=len(concat_rows))
year_series = pd.Series(years, name='Year')

# Объединяем с датафреймом и называем результат 'fines'
fines = pd.concat([concat_rows, year_series], axis=1)
print("\nДатафрейм с добавленной колонкой Year:")
print(fines.head())


Размер объединенного датафрейма: (925, 5)

Датафрейм с добавленной колонкой Year:
      CarNumber  Refund   Fines    Make  Model  Year
0  Y163O8161RUS    2.00 3200.00    Ford  Focus  1989
1   E432XX77RUS    1.00 6500.00  Toyota  Camry  1995
2   7184TT36RUS    1.00 2100.00    Ford  Focus  1984
3  X582HE161RUS    2.00 2000.00    Ford  Focus  2015
4  92918M178RUS    1.00 5700.00    Ford  Focus  2014


In [4]:
# Читаем данные о фамилиях из JSON
with open('../../datasets/surname.json', 'r') as f:
    surname_data = json.load(f)

# Преобразуем в DataFrame и очищаем от специальных символов
surnames_df = pd.DataFrame(surname_data[1:], columns=surname_data[0])  # Пропускаем заголовок
surnames_list = [name.strip('",[]') for name in surnames_df['NAME'].values]

print(f"\nКоличество фамилий: {len(surnames_list)}")
print("Первые 10 фамилий:", surnames_list[:10])


Количество фамилий: 100
Первые 10 фамилий: ['ADAMS', 'ALLEN', 'ALVAREZ', 'ANDERSON', 'BAILEY', 'BAKER', 'BENNETT', 'BROOKS', 'BROWN', 'CAMPBELL']


In [5]:
# Создаем датафрейм owners
unique_car_numbers = fines['CarNumber'].unique()
num_unique_cars = len(unique_car_numbers)

# Выбираем случайные фамилии с random_state=21
random.seed(21)
selected_surnames = random.choices(surnames_list, k=num_unique_cars)

owners = pd.DataFrame({
    'CarNumber': unique_car_numbers,
    'SURNAME': selected_surnames
})

print(f"\nДатафрейм owners создан с {len(owners)} записями:")
print(owners.head())


Датафрейм owners создан с 531 записями:
      CarNumber  SURNAME
0  Y163O8161RUS   COOPER
1   E432XX77RUS    PRICE
2   7184TT36RUS   PARKER
3  X582HE161RUS     LONG
4  92918M178RUS  EDWARDS


In [6]:
additional_fines = pd.DataFrame({
    'CarNumber': ['NEW001RUS', 'NEW002RUS', 'NEW003RUS', 'NEW004RUS', 'NEW005RUS'],
    'Refund': [2.0, 1.5, 2.5, 1.0, 2.0],
    'Fines': [5000.0, 3000.0, 7000.0, 2000.0, 4000.0],
    'Make': ['BMW', 'Audi', 'Mercedes', 'Lexus', 'Volvo'],
    'Model': ['X5', 'A4', 'C-Class', 'RX', 'XC90'],
    'Year': [2015, 2016, 2017, 2018, 2019]
})

fines = pd.concat([fines, additional_fines], ignore_index=True)
print(f"\nРазмер fines после добавления 5 записей: {fines.shape}")


Размер fines после добавления 5 записей: (930, 6)


In [7]:
# Удаляем последние 20 записей из owners и добавляем 3 новые
owners = owners.iloc[:-20].copy()

additional_owners = pd.DataFrame({
    'CarNumber': ['OWNER001RUS', 'OWNER002RUS', 'OWNER003RUS'],
    'SURNAME': ['PETROV', 'SIDOROV', 'FEDOROV']
})

owners = pd.concat([owners, additional_owners], ignore_index=True)
print(f"\nРазмер owners после изменений: {owners.shape}")


Размер owners после изменений: (514, 2)


In [8]:
#Различные типы объединений датафреймов
# Inner join - только номера машин, которые есть в обоих датафреймах

inner_join = pd.merge(fines, owners, on='CarNumber', how='inner')
print(f"\nInner join (пересечение): {inner_join.shape[0]} записей")
print("Пример записей:")
print(inner_join[['CarNumber', 'Make', 'Model', 'Fines', 'SURNAME']].head())

# Outer join - все номера машин из обоих датафреймов
outer_join = pd.merge(fines, owners, on='CarNumber', how='outer')
print(f"\nOuter join (объединение): {outer_join.shape[0]} записей")

# Left join - только номера машин из fines
left_join = pd.merge(fines, owners, on='CarNumber', how='left')
print(f"\nLeft join (все из fines): {left_join.shape[0]} записей")

# Right join - только номера машин из owners
right_join = pd.merge(fines, owners, on='CarNumber', how='right')
print(f"\nRight join (все из owners): {right_join.shape[0]} записей")


Inner join (пересечение): 897 записей
Пример записей:
      CarNumber    Make  Model   Fines  SURNAME
0  Y163O8161RUS    Ford  Focus 3200.00   COOPER
1   E432XX77RUS  Toyota  Camry 6500.00    PRICE
2   7184TT36RUS    Ford  Focus 2100.00   PARKER
3  X582HE161RUS    Ford  Focus 2000.00     LONG
4  92918M178RUS    Ford  Focus 5700.00  EDWARDS

Outer join (объединение): 933 записей

Left join (все из fines): 930 записей

Right join (все из owners): 900 записей


In [9]:
# Создаем сводную таблицу
# Создаем колонку Make_Model для сводной таблицы
fines['Make_Model'] = fines['Make'] + ' ' + fines['Model'].fillna('Unknown')

# Создаем сводную таблицу
pivot_table = pd.pivot_table(
    fines,
    values='Fines',
    index='Make_Model',
    columns='Year',
    aggfunc='sum'
)

print("Сводная таблица (сумма штрафов по моделям и годам):")
print(pivot_table.iloc[:10, :10])  # Показываем первые 10 строк и 10 столбцов

# Показываем размеры сводной таблицы
print(f"\nРазмер сводной таблицы: {pivot_table.shape}")
print(f"Годы в сводной таблице: {sorted(pivot_table.columns.tolist())}")

# Показываем модели с наибольшими штрафами
total_fines_by_model = pivot_table.sum(axis=1).sort_values(ascending=False)
print(f"\nТоп-10 моделей по общей сумме штрафов:")
print(total_fines_by_model.head(10))

Сводная таблица (сумма штрафов по моделям и годам):
Year                 1980      1981      1982     1983     1984      1985  \
Make_Model                                                                  
Audi                  NaN       NaN       NaN      NaN      NaN       NaN   
Audi A4               NaN       NaN       NaN      NaN      NaN       NaN   
BMW                   NaN       NaN       NaN      NaN      NaN       NaN   
BMW X5                NaN       NaN       NaN      NaN      NaN       NaN   
Ford Focus       92889.17 395483.76 166283.76 85800.00 99489.17 155178.35   
Ford Mondeo           NaN       NaN       NaN      NaN      NaN       NaN   
Lexus RX              NaN       NaN       NaN      NaN      NaN       NaN   
Mercedes C-Class      NaN       NaN       NaN      NaN      NaN       NaN   
Skoda Octavia     1900.00       NaN   6900.00 11594.59      NaN  10294.59   
Toyota Camry     12500.00   8594.59       NaN  7200.00      NaN       NaN   

Year                  1

In [10]:

# 11. Сохраняем датафреймы в CSV файлы без индекса
try:
    fines.to_csv('fines.csv', index=False)
    owners.to_csv('owners.csv', index=False)
    print("Файлы успешно сохранены:")
    print("- fines.csv")
    print("- owners.csv")

except Exception as e:
    print("Произошла ошибка при создании файлов")
    raise e

Файлы успешно сохранены:
- fines.csv
- owners.csv


In [11]:
# Финальная информация о датафреймах
print(f"Размер датафрейма fines: {fines.shape}")
print(f"Размер датафрейма owners: {owners.shape}")
print(f"Уникальных номеров машин в fines: {fines['CarNumber'].nunique()}")
print(f"Уникальных номеров машин в owners: {owners['CarNumber'].nunique()}")

print("\nПример записей из финального датафрейма fines:")
print(fines.tail())

print("\nПример записей из датафрейма owners:")
print(owners.tail())


Размер датафрейма fines: (930, 7)
Размер датафрейма owners: (514, 2)
Уникальных номеров машин в fines: 536
Уникальных номеров машин в owners: 514

Пример записей из финального датафрейма fines:
     CarNumber  Refund   Fines      Make    Model  Year        Make_Model
925  NEW001RUS    2.00 5000.00       BMW       X5  2015            BMW X5
926  NEW002RUS    1.50 3000.00      Audi       A4  2016           Audi A4
927  NEW003RUS    2.50 7000.00  Mercedes  C-Class  2017  Mercedes C-Class
928  NEW004RUS    1.00 2000.00     Lexus       RX  2018          Lexus RX
929  NEW005RUS    2.00 4000.00     Volvo     XC90  2019        Volvo XC90

Пример записей из датафрейма owners:
        CarNumber  SURNAME
509  O50197197RUS   MURPHY
510  7608EE777RUS    BROWN
511   OWNER001RUS   PETROV
512   OWNER002RUS  SIDOROV
513   OWNER003RUS  FEDOROV
